# Battle Of Neighborhood - Bangalore

#### Import all the required Libraries.

In [119]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from bs4 import BeautifulSoup
import requests
import csv

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge geocoder --yes # install geocoder
import geocoder

print('Libraries imported.')

Libraries imported.


#### Get the Postal Code and Neighborhoods of Bangalore from the mapsofindia website using beatifulsoup

In [120]:
source = requests.get('https://www.mapsofindia.com/pincode/india/karnataka/bangalore/').text

bsoup = BeautifulSoup(source,'lxml')

tablesoup = bsoup.find('table', class_ = 'extrtable')
#print(tablesoup.prettify())

In [121]:
# Create a CSV File by iterating the required tags to get the Pincode & Neighborhood

bangalore_csv = open('bangalore_neighborhood.csv', 'w')
csv_writer = csv.writer(bangalore_csv)
csv_writer.writerow(['Neighborhood','Pincode']) 

for tr in tablesoup.find_all('tr'):
#    print(tr)   
#tr = tablesoup.find('tr')
#print(tr.find('a'))
    if tr.find('a') != None :
        Neighborhood = tr.find('a').text
        Pincode = tr.find('b').text.split()[0]
#        print(Neighborhood, Pincode)
        csv_writer.writerow([Neighborhood,Pincode]) 

bangalore_csv.close()

In [122]:
#Create the dataframe from the CSV
df_b = pd.read_csv('bangalore_neighborhood.csv')

df_b.head()

,Neighborhood,Pincode
0,A F station yelahanka,560063
1,Adugodi,560030
2,Agara,560034
3,Agram,560007
4,Air Force hospital,560007


In [123]:
df_b.shape

(294, 2)

In [124]:
# Neighborhoods with same Pincode are combined with a comma

grouped = df_b.groupby(['Pincode'])['Neighborhood'].apply(', '.join)

groupedDF = grouped.to_frame().reset_index()

print(groupedDF.shape)
groupedDF.head()

(99, 2)


,Pincode,Neighborhood
0,560001,"Bangalore Bazaar, Bangalore., Brigade Road, Cu..."
1,560002,"Avenue Road, Bangalore City, Bangalore Corpora..."
2,560003,"Aranya Bhavan, Malleswaram, Palace Guttahalli,..."
3,560004,"Basavanagudi, Lalbagh West, Mavalli, Pasmpamah..."
4,560005,"Fraser Town, Jeevanahalli"


### This code was used to retrieve lat,lon of all the postal code for one time due to limitation of google gecoder usage and saved the data in a CSV for reuse.

import geocoder # import geocoder

bangalore_LLcsv = open('bangalore_LL.csv', 'w')
csv_writer = csv.writer(bangalore_LLcsv)
csv_writer.writerow(['Pincode','latitude','longitude']) 

for index, row in groupedDF.iterrows():
    # initialize your variable to None
    lat_lng_coords = None
    postalcode = groupedDF.loc[index,'Pincode']
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Bangalore, Karnataka'.format(postalcode))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    #print(postalcode, '-> ', latitude, longitude)
    csv_writer.writerow([postalcode,latitude,longitude])

bangalore_LLcsv.close()
print('ll Completed')

In [125]:
# Create a DataFrame for the Lat, Lon data from the saved CSV File
bdf_geoloc = pd.read_csv('Bangalore_LL.csv')
bdf_geoloc.columns=['Pincode','Latitude','Longitude']
bdf_geoloc.head()

,Pincode,Latitude,Longitude
0,560001,12.976594,77.599271
1,560002,12.963521,77.582106
2,560003,13.001914,77.571336
3,560004,12.943751,77.573760
4,560005,12.997989,77.622651


In [126]:
# Merge the 2 dataframe comparing the Postalcode column
bdf_latlon = pd.merge(groupedDF, bdf_geoloc, on='Pincode')

bdf_latlon.head(10)

,Pincode,Neighborhood,Latitude,Longitude
0,560001,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.976594,77.599271
1,560002,"Avenue Road, Bangalore City, Bangalore Corpora...",12.963521,77.582106
2,560003,"Aranya Bhavan, Malleswaram, Palace Guttahalli,...",13.001914,77.571336
3,560004,"Basavanagudi, Lalbagh West, Mavalli, Pasmpamah...",12.943751,77.573760
4,560005,"Fraser Town, Jeevanahalli",12.997989,77.622651
5,560006,"J.C.nagar, Training Command iaf",13.013841,77.587667
6,560007,"Agram, Air Force hospital",12.957917,77.630912
7,560008,"H.A.l ii stage, Hulsur Bazaar, Someswarapura",12.981994,77.625613
8,560009,"Bangalore Dist offices bldg, K. g. road, Subha...",12.977029,77.577305
9,560010,"Bhashyam Circle, Industrial Estate, Rajajinaga...",12.984981,77.553344


#### Use geopy library to get the latitude and longitude values of Bangalore.

In [127]:
address = 'Bangalore, India'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Bangalore are 12.9791198, 77.5912997.


### Show the Neighborhoods in Bangalore Map

In [128]:
# create map of Bangalore using latitude and longitude values

map_bangalore = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(bdf_latlon['Latitude'], bdf_latlon['Longitude'], bdf_latlon['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bangalore)  
    
map_bangalore

### Define Foursquare Credentials and Version

In [129]:
CLIENT_ID = 'KAK50CKL4R2TKOFNRTELIGQRHURR2EKB00DK2WV1UM3TEZBP' # your Foursquare ID
CLIENT_SECRET = 'JKSD5YXIQSPBNSYEVOQW0ZFIVT1OHMX12IJVWJWREED5SUPP' # your Foursquare Secret
VERSION = '20180930' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KAK50CKL4R2TKOFNRTELIGQRHURR2EKB00DK2WV1UM3TEZBP
CLIENT_SECRET:JKSD5YXIQSPBNSYEVOQW0ZFIVT1OHMX12IJVWJWREED5SUPP


#### Let's explore the first neighborhood in our dataframe.

In [130]:
bdf_latlon.loc[0, 'Neighborhood']

'Bangalore Bazaar, Bangalore., Brigade Road, Cubban Road, Dr. ambedkar veedhi, Highcourt, Legislators Home, Mahatma Gandhi road, Narayan Pillai street, Rajbhavan, Vidhana Soudha'

In [131]:
neighborhood_latitude = bdf_latlon.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = bdf_latlon.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood = bdf_latlon.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bangalore Bazaar, Bangalore., Brigade Road, Cubban Road, Dr. ambedkar veedhi, Highcourt, Legislators Home, Mahatma Gandhi road, Narayan Pillai street, Rajbhavan, Vidhana Soudha are 12.9765944, 77.5992708.


#### Now, let's get the top 100 venues that are in the 1st Neighborhood within a radius of 1000 meters.

In [132]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=KAK50CKL4R2TKOFNRTELIGQRHURR2EKB00DK2WV1UM3TEZBP&client_secret=JKSD5YXIQSPBNSYEVOQW0ZFIVT1OHMX12IJVWJWREED5SUPP&v=20180930&ll=12.9765944,77.5992708&radius=1000&limit=100'

Send the GET request and examine the resutls

In [133]:
results = requests.get(url).json()
results
type(results)

dict

In [134]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [135]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,M. Chinnaswamy Stadium,Cricket Ground,12.978144,77.599223
1,Hard Rock Cafe Bengaluru,American Restaurant,12.976154,77.601616
2,Corner House,Ice Cream Shop,12.972979,77.599971
3,Cubbon Park,Park,12.977042,77.595277
4,Rice Bowl,Chinese Restaurant,12.974596,77.599470


In [136]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


### Now creating a function to repeat the same process to all the Nieghborhoods in Bangalore

In [137]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Call the Function to parse each neighborhood and create the bangalore_venues DF

In [138]:
bangalore_venues = getNearbyVenues(names=bdf_latlon['Neighborhood'],
                                   latitudes=bdf_latlon['Latitude'],
                                   longitudes=bdf_latlon['Longitude']
                                  )

Bangalore Bazaar, Bangalore., Brigade Road, Cubban Road, Dr. ambedkar veedhi, Highcourt, Legislators Home, Mahatma Gandhi road, Narayan Pillai street, Rajbhavan, Vidhana Soudha
Avenue Road, Bangalore City, Bangalore Corporation building, Bangalore Fort, Basavaraja Market, Cahmrajendrapet, Narasimjharaja Road, New Tharaggupet, Sri Jayachamarajendra road
Aranya Bhavan, Malleswaram, Palace Guttahalli, Swimming Pool extn, Venkatarangapura, Vyalikaval Extn
Basavanagudi, Lalbagh West, Mavalli, Pasmpamahakavi Road, Shankarpura, Visveswarapuram
Fraser Town, Jeevanahalli
J.C.nagar, Training Command iaf
Agram, Air Force hospital
H.A.l ii stage, Hulsur Bazaar, Someswarapura
Bangalore Dist offices bldg, K. g. road, Subhashnagar
Bhashyam Circle, Industrial Estate, Rajajinagar, Rajajinagar I block, Rajajinagar Ivth block
Jayangar Iii block, Madhavan Park
Science Institute
Govindapalya, H M t, Jalahalli, Jalahalli Village
Jalahalli East
Jalahalli West, Kamagondanahalli
Doorvaninagar, Krishnarajapuram

KeyError: 'groups'

In [139]:
print(bangalore_venues.shape)
bangalore_venues.head()

(2366, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.976594,77.599271,M. Chinnaswamy Stadium,12.978144,77.599223,Cricket Ground
1,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.976594,77.599271,Hard Rock Cafe Bengaluru,12.976154,77.601616,American Restaurant
2,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.976594,77.599271,Corner House,12.972979,77.599971,Ice Cream Shop
3,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.976594,77.599271,Cubbon Park,12.977042,77.595277,Park
4,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.976594,77.599271,Rice Bowl,12.974596,77.599470,Chinese Restaurant


Let's check how many venues were returned for each neighborhood

In [140]:
bangalore_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
A F station yelahanka,5,5,5,5,5,5
"Adugodi, Hosur Road",9,9,9,9,9,9
"Agara, Kendriya Sadan, Koramangala, Koramangala I block, St. john's medical college",56,56,56,56,56,56
"Agram, Air Force hospital",4,4,4,4,4,4
"Amruthahalli, Kodigehalli, Sahakaranagar P.o",27,27,27,27,27,27
"Anandnagar, H.A. farm, Hebbal Kempapura",14,14,14,14,14,14
"Anekal, Anekalbazar, Bestamaranahalli, Harogadde, Hennagara, Hulimangala, Indalavadi, Jigani, Marsur, Samandur, Sidihoskote, Thammanayakanahalli, Vanakanahalli",1,1,1,1,1,1
"Arabic College, Devarjeevanahalli, Nagavara, Ramakrishna Hegde nagar, Venkateshapura",5,5,5,5,5,5
"Aranya Bhavan, Malleswaram, Palace Guttahalli, Swimming Pool extn, Venkatarangapura, Vyalikaval Extn",51,51,51,51,51,51


### Let's find out how many unique categories can be curated from all the returned venues

In [141]:
print('There are {} uniques categories.'.format(len(bangalore_venues['Venue Category'].unique())))

There are 206 uniques categories.


### Let's find out the top 10 Venue Categories in the neighborhoods

In [142]:
bangalore_venues['Venue Category'].value_counts().head(10)

Indian Restaurant       428
Café                    161
Fast Food Restaurant     85
Ice Cream Shop           72
Pizza Place              70
Department Store         70
Coffee Shop              68
Hotel                    64
Bakery                   59
Chinese Restaurant       58
Name: Venue Category, dtype: int64

### Analyze Each Neighborhood

In [143]:
# one hot encoding
bangalore_onehot = pd.get_dummies(bangalore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bangalore_onehot['Neighborhood'] = bangalore_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bangalore_onehot.columns[-1]] + list(bangalore_onehot.columns[:-1])
bangalore_onehot = bangalore_onehot[fixed_columns]

bangalore_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hot Spring,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Nightclub,North Indian Restaurant,Other Nightlife,Outdoors & Recreation,Outlet Store,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Radio Station,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [144]:
bangalore_onehot.shape

(2366, 207)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [145]:
bangalore_grouped = bangalore_onehot.groupby('Neighborhood').mean().reset_index()
bangalore_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hot Spring,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Nightclub,North Indian Restaurant,Other Nightlife,Outdoors & Recreation,Outlet Store,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Radio Station,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,A F station yelahanka,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.2,0.000000,0.000000,0.000000,0.000000,0.400000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,"Adugodi, Ho

In [146]:
bangalore_grouped.shape

(96, 207)

## Create the bangalore_data Dataframe Removing the Neighbors where there were no venues found

In [147]:
bangalore_data = pd.merge(bangalore_grouped[['Neighborhood']], bdf_latlon, on='Neighborhood')

print(bangalore_data.shape)
bangalore_data.head()

(96, 4)


,Neighborhood,Pincode,Latitude,Longitude
0,A F station yelahanka,560063,13.129087,77.614226
1,"Adugodi, Hosur Road",560030,12.944150,77.607623
2,"Agara, Kendriya Sadan, Koramangala, Koramangal...",560034,12.926138,77.622109
3,"Agram, Air Force hospital",560007,12.957917,77.630912
4,"Amruthahalli, Kodigehalli, Sahakaranagar P.o",560092,13.064104,77.593121


### Let's print each neighborhood along with the top 5 most common venues

In [148]:
num_top_venues = 5

for hood in bangalore_grouped['Neighborhood']:
    print("----",hood,"----")
    temp = bangalore_grouped[bangalore_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- A F station yelahanka ----
               venue  freq
0  Indian Restaurant   0.4
1   Department Store   0.2
2         Hot Spring   0.2
3               Lake   0.2
4  Accessories Store   0.0


---- Adugodi, Hosur Road ----
               venue  freq
0               Café  0.33
1                Spa  0.11
2       Dessert Shop  0.11
3  Kerala Restaurant  0.11
4        Bus Station  0.11


---- Agara, Kendriya Sadan, Koramangala, Koramangala I block, St. john's medical college ----
               venue  freq
0  Indian Restaurant  0.21
1               Café  0.16
2        Coffee Shop  0.05
3     Ice Cream Shop  0.05
4             Bakery  0.05


---- Agram, Air Force hospital ----
               venue  freq
0  Indian Restaurant  0.50
1        Pizza Place  0.25
2      Historic Site  0.25
3  Accessories Store  0.00
4  Paella Restaurant  0.00


---- Amruthahalli, Kodigehalli, Sahakaranagar P.o ----
               venue  freq
0  Indian Restaurant  0.22
1        Bus Station  0.11
2     Ice Cream 

### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.


In [149]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [150]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bangalore_grouped['Neighborhood']

for ind in np.arange(bangalore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bangalore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A F station yelahanka,Indian Restaurant,Hot Spring,Department Store,Lake,Eastern European Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant
1,"Adugodi, Hosur Road",Café,Spa,Dessert Shop,Bus Station,Kerala Restaurant,Restaurant,Business Service,Electronics Store,Flea Market,Fish & Chips Shop
2,"Agara, Kendriya Sadan, Koramangala, Koramangal...",Indian Restaurant,Café,Bakery,Ice Cream Shop,Coffee Shop,Fast Food Restaurant,Pub,Hotel,Pizza Place,Mexican Restaurant
3,"Agram, Air Force hospital",Indian Restaurant,Pizza Place,Historic Site,Dumpling Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
4,"Amruthahalli, Kodigehalli, Sahakaranagar P.o",Indian Restaurant,Ice Cream Shop,Bus Station,Resort,Brewery,Hotel,Italian Restaurant,Flea Market,Fast Food Restaurant,Department Store
5,"Anandnagar, H.A. farm, Hebbal Kempapura",Indian Restaurant,Bakery,Pizza Place,Burger Joint,Lake,Department Store,Pharmacy,Café,Bus Station,Ice Cream Shop
6,"Anekal, Anekalbazar, Bestamaranahalli, Harogad...",Business Service,Yoga Studio,Convenience Store,Food,Flower Shop,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
7,"Arabic College, Devarjeevanahalli, Nagavara, R...",Indian Restaurant,Coffee Shop,Pharmacy,Shopping Mall,Auto Workshop,Eastern European Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Field
8,"Aranya Bhavan, Malleswaram, Palace Guttahalli,...",Indian Restaurant,Ice Cream Shop,Vegetarian / Vegan Restaurant,Café,Dessert Shop,Snack Place,Breakfast Spot,Chinese Restaurant,Plaza,Coffee Shop
9,"Ashoknagar, Banashankari, Dasarahalli, State B...",Snack Place,Indian Restaurant,Department Store,Fast Food Restaurant,Breakfast Spot,Park,Vegetarian / Vegan Restaurant,Juice Bar,Arcade,Bakery


## Cluster Neighborhoods

#### Run k-means to cluster the neighborhood into 4 clusters.


In [151]:
# set number of clusters
kclusters = 4

bangalore_grouped_clustering = bangalore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bangalore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 0, 0, 2, 1, 0, 0, 2], dtype=int32)

In [152]:
len(kmeans.labels_)

96

In [153]:
bangalore_grouped_clustering.head()

,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hot Spring,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Nightclub,North Indian Restaurant,Other Nightlife,Outdoors & Recreation,Outlet Store,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Radio Station,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.2,0.000000,0.000000,0.000000,0.000000,0.400000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [154]:
bangalore_merged = bangalore_data

# add clustering labels
bangalore_merged['Cluster Labels'] = kmeans.labels_

# merge bangalore_grouped with bangalore_data to add latitude/longitude for each neighborhood
bangalore_merged = bangalore_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'),
                                                                on='Neighborhood')

bangalore_merged.head() # check the last columns!

,Neighborhood,Pincode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A F station yelahanka,560063,13.129087,77.614226,0,Indian Restaurant,Hot Spring,Department Store,Lake,Eastern European Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant
1,"Adugodi, Hosur Road",560030,12.944150,77.607623,2,Café,Spa,Dessert Shop,Bus Station,Kerala Restaurant,Restaurant,Business Service,Electronics Store,Flea Market,Fish & Chips Shop
2,"Agara, Kendriya Sadan, Koramangala, Koramangal...",560034,12.926138,77.622109,0,Indian Restaurant,Café,Bakery,Ice Cream Shop,Coffee Shop,Fast Food Restaurant,Pub,Hotel,Pizza Place,Mexican Restaurant
3,"Agram, Air Force hospital",560007,12.957917,77.630912,0,Indian Restaurant,Pizza Place,Historic Site,Dumpling Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
4,"Amruthahalli, Kodigehalli, Sahakaranagar P.o",560092,13.064104,77.593121,0,Indian Restaurant,Ice Cream Shop,Bus Station,Resort,Brewery,Hotel,Italian Restaurant,Flea Market,Fast Food Restaurant,Department Store


In [155]:
bangalore_merged['Cluster Labels'].value_counts()

2    50
0    44
3     1
1     1
Name: Cluster Labels, dtype: int64

### Finally, let's visualize the resulting clusters

In [156]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bangalore_merged['Latitude'], bangalore_merged['Longitude'], bangalore_merged['Neighborhood'], bangalore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-2],
        fill_opacity=0.5).add_to(map_clusters)
       
map_clusters

## Lets Evaluate the clusters and find out potential business point for a new Restaurant

### Cluster-0

In [157]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 0,
                     #bangalore_merged.columns[['Pincode']],
                     bangalore_merged.columns[[0]+[1] + list(range(4, bangalore_merged.shape[1]))]]

,Neighborhood,Pincode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A F station yelahanka,560063,0,Indian Restaurant,Hot Spring,Department Store,Lake,Eastern European Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant
2,"Agara, Kendriya Sadan, Koramangala, Koramangal...",560034,0,Indian Restaurant,Café,Bakery,Ice Cream Shop,Coffee Shop,Fast Food Restaurant,Pub,Hotel,Pizza Place,Mexican Restaurant
3,"Agram, Air Force hospital",560007,0,Indian Restaurant,Pizza Place,Historic Site,Dumpling Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
4,"Amruthahalli, Kodigehalli, Sahakaranagar P.o",560092,0,Indian Restaurant,Ice Cream Shop,Bus Station,Resort,Brewery,Hotel,Italian Restaurant,Flea Market,Fast Food Restaurant,Department Store
7,"Arabic College, Devarjeevanahalli, Nagavara, R...",560045,0,Indian Restaurant,Coffee Shop,Pharmacy,Shopping Mall,Auto Workshop,Eastern European Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Field
8,"Aranya Bhavan, Malleswaram, Palace Guttahalli,...",560003,0,Indian Restaurant,Ice Cream Shop,Vegetarian / Vegan Restaurant,Café,Dessert Shop,Snack Place,Breakfast Spot,Chinese Restaurant,Plaza,Coffee Shop
12,"Avalahalli, Bapujinagar, Byatarayanapura, Deep...",560026,0,Indian Restaurant,Pizza Place,Department Store,Fast Food Restaurant,Bus Station,Café,Electronics Store,Flower Shop,Flea Market,Fish & Chips Shop
14,"Avenue Road, Bangalore City, Bangalore Corpora...",560002,0,Indian Restaurant,Theater,South Indian Restaurant,Historic Site,Food Truck,Miscellaneous Shop,Movie Theater,Platform,Hotel,Rajasthani Restaurant
16,"Bagalgunte, Nagasandra",560073,0,Indian Restaurant,Light Rail Station,Gas Station,Athletics & Sports,Flower Shop,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
18,"Balepete, Chickpet",560053,0,Indian Restaurant,Bookstore,Hotel,Middle Eastern Restaurant,Restaurant,Miscellaneous Shop,Flea Market,South Indian Restaurant,Food Truck,Seafood Restaurant


### Cluster-1

In [158]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 1, 
                     bangalore_merged.columns[[0]+[1] + list(range(5, bangalore_merged.shape[1]))]]

,Neighborhood,Pincode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"Anekal, Anekalbazar, Bestamaranahalli, Harogad...",562106,Business Service,Yoga Studio,Convenience Store,Food,Flower Shop,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market


### Cluster-2

In [159]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 2, 
                     bangalore_merged.columns[[0]+[1] + list(range(5, bangalore_merged.shape[1]))]]

,Neighborhood,Pincode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Adugodi, Hosur Road",560030,Café,Spa,Dessert Shop,Bus Station,Kerala Restaurant,Restaurant,Business Service,Electronics Store,Flea Market,Fish & Chips Shop
5,"Anandnagar, H.A. farm, Hebbal Kempapura",560024,Indian Restaurant,Bakery,Pizza Place,Burger Joint,Lake,Department Store,Pharmacy,Café,Bus Station,Ice Cream Shop
9,"Ashoknagar, Banashankari, Dasarahalli, State B...",560050,Snack Place,Indian Restaurant,Department Store,Fast Food Restaurant,Breakfast Spot,Park,Vegetarian / Vegan Restaurant,Juice Bar,Arcade,Bakery
10,"Attur, Bsf Campus yelahanka, Crpf Campus yelah...",560064,Café,Indian Restaurant,Bakery,Chinese Restaurant,Department Store,Restaurant,Smoke Shop,South Indian Restaurant,Food Truck,Soccer Field
11,"Austin Town, Viveknagar",560047,Spa,Park,Fast Food Restaurant,Café,Indie Movie Theater,Cricket Ground,Eastern European Restaurant,Flower Shop,Flea Market,Fish & Chips Shop
13,"Avani Sringeri mutt, Basaveswaranagar Ii stage...",560086,Bakery,Pizza Place,Indian Restaurant,Gym / Fitness Center,Food Truck,Middle Eastern Restaurant,Food & Drink Shop,Fast Food Restaurant,Electronics Store,Park
15,"B Sk ii stage, Jayanagar West, Padmanabhnagar,...",560070,Café,Indian Restaurant,Snack Place,Gym,Fast Food Restaurant,Department Store,Convenience Store,Hotel Bar,Sandwich Place,Seafood Restaurant
17,"Bagalur, Bandikodigehalli, Doddagubbi, Kannur",562149,Sports Club,Yoga Studio,Food & Drink Shop,Flower Shop,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Event Space
19,"Banashankari Iii stage, Girinagar, Hosakerehal...",560085,Clothing Store,Juice Bar,Pizza Place,Café,Indian Restaurant,Breakfast Spot,Fried Chicken Joint,Miscellaneous Shop,Fast Food Restaurant,North Indian Restaurant
20,"Banaswadi, Horamavu, Jalavayuvihar, Kalyanagar",560043,Café,Indian Restaurant,Department Store,Ice Cream Shop,BBQ Joint,Bistro,Korean Restaurant,Pub,Fast Food Restaurant,Brewery


### Cluster-3

In [160]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 3, 
                     bangalore_merged.columns[[0]+[1] + list(range(5, bangalore_merged.shape[1]))]]

,Neighborhood,Pincode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Bannerghatta,560083,Resort,Yoga Studio,Dumpling Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Event Space


## Observation & Evaluation ->

### Cluster 1 & 3 -                
            There are only one 'Pincode Area' in each of these 2 clusters and they are
            looking perfect for opening a restaurant as there are very less no. of 
            restaurants and other places like business areas and flea markets which
            are indication of potential customer based areas for restaurant.
            But this is good if someone is looking to open a restaurants in outskirts
            of bangalore as these locations are very far from the actual city market area.
### Cluster 0 -
            These neighborhoods have lot of restaurants and eateries. Although these 
            places would have high business turn around thats why there are so many 
            eateries already established, but competition will be very very tough 
            for a new restaurant to open and settle.
### Cluster 2 -
            These neighborhoods have mix of all types of venues, lets explore and analyse
            which areas have less restaurants and eateries and other business venues which
            will fetch potential customers for a restaurant.

### Lets create the Cluster2 dataframce and analyze further

In [161]:
cl2 = bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 2, 
                     bangalore_merged.columns[[0]+[1]+[2]+[3] + list(range(5,bangalore_merged.shape[1]))]]

cl2.reset_index(drop=True, inplace = True)

print(cl2.shape)
cl2.head()

(50, 14)


,Neighborhood,Pincode,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adugodi, Hosur Road",560030,12.944150,77.607623,Café,Spa,Dessert Shop,Bus Station,Kerala Restaurant,Restaurant,Business Service,Electronics Store,Flea Market,Fish & Chips Shop
1,"Anandnagar, H.A. farm, Hebbal Kempapura",560024,13.048745,77.592330,Indian Restaurant,Bakery,Pizza Place,Burger Joint,Lake,Department Store,Pharmacy,Café,Bus Station,Ice Cream Shop
2,"Ashoknagar, Banashankari, Dasarahalli, State B...",560050,12.935307,77.558531,Snack Place,Indian Restaurant,Department Store,Fast Food Restaurant,Breakfast Spot,Park,Vegetarian / Vegan Restaurant,Juice Bar,Arcade,Bakery
3,"Attur, Bsf Campus yelahanka, Crpf Campus yelah...",560064,13.101970,77.586359,Café,Indian Restaurant,Bakery,Chinese Restaurant,Department Store,Restaurant,Smoke Shop,South Indian Restaurant,Food Truck,Soccer Field
4,"Austin Town, Viveknagar",560047,12.953679,77.617625,Spa,Park,Fast Food Restaurant,Café,Indie Movie Theater,Cricket Ground,Eastern European Restaurant,Flower Shop,Flea Market,Fish & Chips Shop


#### List out neighborhoods don't have "Indian Restaurant" in their 10 most common places
As we have seen in the cluster-1 * as well as in the cluster-2, the venue category
"Indian Restaurant" is most popular and common places in most of the neighborhoods. So Let's find out the neighborhoods where there are no "Indian Restaurant" venues in the 10 most common venues.

In [162]:
cl2noIR = cl2[cl2.apply(lambda r: r.str.contains('Indian Restaurant', case=False).any(), axis=1) == False]

print('We found {} neighboorhood areas not having Indian Restaurant as their 10 most common venues'.
      format(cl2noIR.shape[0]))

cl2noIR.reset_index(drop=True, inplace=True)
cl2noIR.head()

We found 18 neighboorhood areas not having Indian Restaurant as their 10 most common venues


,Neighborhood,Pincode,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adugodi, Hosur Road",560030,12.944150,77.607623,Café,Spa,Dessert Shop,Bus Station,Kerala Restaurant,Restaurant,Business Service,Electronics Store,Flea Market,Fish & Chips Shop
1,"Austin Town, Viveknagar",560047,12.953679,77.617625,Spa,Park,Fast Food Restaurant,Café,Indie Movie Theater,Cricket Ground,Eastern European Restaurant,Flower Shop,Flea Market,Fish & Chips Shop
2,"Bagalur, Bandikodigehalli, Doddagubbi, Kannur",562149,13.119511,77.678614,Sports Club,Yoga Studio,Food & Drink Shop,Flower Shop,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Event Space
3,"Chikkalasandra, Subramanyapura",560061,12.903725,77.537639,Movie Theater,Café,Restaurant,Dumpling Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
4,"Devanagundi, Kadugodi, Kalkunte, Kannamangala,...",560067,12.996701,77.758197,Hotel,Pizza Place,Train Station,Bus Station,Yoga Studio,Dumpling Restaurant,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant


#### Now let's get the density of the venue category "Restaurant" for each neighborhood
    For this we will check common venue categories for each neighborhood, if its any 
    kind of Restaurant then multiply it with its corresponding weightage value 
    (1st common venue = 10 & 10th common venue = 1)then sum up all the weightage values.

In [163]:
rDensityList = []
for index, row in cl2noIR.iterrows():
    counter = 10
    restaurantDensity = 0
    for col in cl2noIR.iloc[:,4:15]:
#        print(counter, cl2noIR.loc[index,col])
        venue_type = cl2noIR.loc[index,col]
        if 'Restaurant' in venue_type:
            restaurantDensity += counter
        counter -= 1
#    print('\n', "Restaurant Weight ->", restaurantDensity, '\n')
    rDensityList.append(restaurantDensity)
rDensityList

[11, 12, 3, 17, 6, 5, 9, 0, 9, 11, 0, 0, 5, 9, 12, 7, 0, 15]

#### Add the Restaurant Density column into the dataframe

In [164]:
cl2noIR['Restaurant Density'] = rDensityList
#cl2noIR.loc[:,'Restaurant Density'] = rDensityList # This is another method
cl2noIR.shape

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(18, 15)

#### Sort the Neighborhoods according to the Restaurant Density

In [165]:
cl2noIR.loc[:,('Neighborhood','Restaurant Density')].sort_values(['Restaurant Density'])

,Neighborhood,Restaurant Density
7,"Gunjur, Muthusandra, Panathur, Vartur",0
16,Nayandahalli,0
10,"Kenchanahalli, Rajarajeshwarinagar",0
11,Koramangala Vi bk,0
2,"Bagalur, Bandikodigehalli, Doddagubbi, Kannur",3
5,"Devasandra, Krishnarajapuram",5
12,Kothanur,5
4,"Devanagundi, Kadugodi, Kalkunte, Kannamangala,...",6
15,Nagarbhavi,7
8,Hampinagar,9


In [166]:
cl2noIR.loc[0,'Restaurant Density']

11

#### Lets create the final dataframe for the recommended neighborhoods with Restaurant Density less than 5 to set up the new Restaurant.

In [176]:
bFinalDF = cl2noIR[cl2noIR[('Restaurant Density')] < 3].sort_values(['Restaurant Density'])
bFinalDF.reset_index(drop=True, inplace=True)

bFinalDF[['Neighborhood','Restaurant Density']]

,Neighborhood,Restaurant Density
0,"Gunjur, Muthusandra, Panathur, Vartur",0
1,"Kenchanahalli, Rajarajeshwarinagar",0
2,Koramangala Vi bk,0
3,Nayandahalli,0


### Plot the Map to show the recomended neighborhoods in the Bangalore Map 

In [178]:
# create map of Bangalore using latitude and longitude values

map_recomend = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood, pincode in zip(bFinalDF['Latitude'], bFinalDF['Longitude'],
                                  bFinalDF['Neighborhood'], bFinalDF['Pincode']):
    label = '{}'', ''{}'.format(neighborhood,pincode)
#    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
#        parse_html=False).add_to(map_recomend)
        ).add_to(map_recomend) 
    
map_recomend

## Final Recomendation of Neighborhoods For A Restaurant & Type of Restaurant
    As per the data available from mapofindia.com website & foursquare API,
    After the plotting the final set of neighborhoods in the Map above, we can confidently say 
    that below two in our list are spot on & are near around the core city as well for opening a
    restaurant
###    1. Koramangala Vi bk 	 - 560095
###    2. Nayandahalli - 560039

### Restaurant Type - Indian Restaurant

     
     
   
### ^^^^^^^^^^^^^^^^    Thank You Very Much Reviewing This    ^^^^^^^^^^^^^^^^